## Cabeçalho
### Objetivo
Limpar/Tratar os dados de proposicoes disponíveis em tb_proposicoes_bronze extraídos da API de dados abertos da Câmara dos Depoutados.

**Sobre
Data de crição**: 30/09/2025

**Responsável:** Wandrieli Nery Barbosa

Parâmetros

In [0]:
%run /Users/wandrieli.89@gmail.com/projeto_tiller/Parametros/nb_parametros

In [0]:
%run /Users/wandrieli.89@gmail.com/projeto_tiller/Parametros/nb_catalag_schema

Transformação

In [0]:
set_catalog_and_schema_silver() #Define catalog e schema conforme run

df_proposicoes_bronze = spark.table("tiller.bronze.tb_proposicoes_bronze") # Carrega a tabela bronze

# Transformações
df_proposicoes_silver = (
     df_proposicoes_bronze
     .withColumn(
        "ementa",
        F.when(F.col("ementa").isNull(), F.lit("Não informado"))
        .otherwise(F.initcap(F.trim(F.col("ementa"))))
    )

    .withColumn(
        "siglaTipo",
        F.when(F.col("siglaTipo").isNull(), F.lit("Não se aplica"))
        .otherwise(F.trim(F.col("siglaTipo")))
    )

    .withColumn("uri", F.trim(F.col("uri")))
    .withColumn("url_origem", F.trim(F.col("url_origem")))

    .withColumn("data_ingestao_silver", F.current_timestamp()) # adiciona coluna de controle da camada silver
)




In [0]:
# Salva a tabela em Delta 
df_proposicoes_silver.write.format("delta").mode("overwrite").saveAsTable("tb_proposicoes_silver")

print("Tabela 'tb_proposicoes_silver' criada/atualizada com sucesso!")

In [0]:
%sql
select * from tiller.silver.tb_proposicoes_silver;